In [3]:
import os
from dotenv import load_dotenv, get_key
load_dotenv()

True

In [17]:
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import os

from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class LoggingCallbackHandler(BaseCallbackHandler):
    def on_chain_start(self, chain, inputs, **kwargs):
        print(f"\n--- Chain Start: {chain.__class__.__name__} ---")
        print(f"Inputs: {inputs}")

    def on_chain_end(self, outputs, **kwargs):
        print(f"\n--- Chain End ---")
        print(f"Outputs: {outputs}")

    def on_llm_start(self, serialized, prompts, **kwargs):
        print("\n--- LLM Start ---")
        print(f"Prompts: {prompts}")

    def on_llm_end(self, response: LLMResult, **kwargs):
        print("\n--- LLM End ---")
        print(f"Response: {response.generations}")
        #for generation in response.generations:
        #    print(f"Response Text: {generation[0].text}")

    def on_llm_error(self, error, **kwargs):
        print("\n--- LLM Error ---")
        print(f"Error: {error}")
        
callback_handler = LoggingCallbackHandler()

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=os.getenv('GEMINI_API_KEY'),
    temperature=0.1
)

abstract_prompt = PromptTemplate(
    input_variables=["idea_description"],
    template=(
        "You are an expert in academic writing. Based on the following idea description, generate a concise and well-structured abstract for an IEEE-style research paper: \n"
        "Idea Description: {idea_description}\n"
        "Abstract:"
    ),
)

section_prompt = PromptTemplate(
    input_variables=["idea_description", "abstract"],
    template=(
        "You are an expert in academic writing. Based on the following idea description and the abstract, generate a detailed structure for an IEEE-style research paper. Include sections: Introduction, Related Work, Problem Statement, Proposed Methodology, Experiments/Results, Conclusion, and Future Work. Provide a concise overview of what each section should contain. \n"
        "Idea Description: {idea_description}\n"
        "Abstract: {abstract}\n"
        "Paper Structure:"
    ),
)

abstract_chain = LLMChain(llm=llm, prompt=abstract_prompt, output_key="abstract")
section_chain = LLMChain(llm=llm, prompt=section_prompt, output_key="paper_structure")

workflow_chain = SequentialChain(
    chains=[abstract_chain, section_chain],
    input_variables=["idea_description"],  # Starting input
    output_variables=["paper_structure"],  # Final output
    verbose=True
)

def generate_paper_structure(idea_description):
    results = workflow_chain.run({"idea_description": idea_description}, callbacks=[callback_handler])
    return results

if __name__ == "__main__":
    idea_description = (
        """A novel idea in wearable technology and security is the BioSecure."""
    )
    output = generate_paper_structure(idea_description)
    print("Final Output:\n", output)



--- Chain Start: NoneType ---
Inputs: {'idea_description': 'A novel idea in wearable technology and security is the BioSecure.'}


> Entering new SequentialChain chain...

--- Chain Start: NoneType ---
Inputs: {'idea_description': 'A novel idea in wearable technology and security is the BioSecure.'}

--- LLM Start ---
Prompts: ['Human: You are an expert in academic writing. Based on the following idea description, generate a concise and well-structured abstract for an IEEE-style research paper: \nIdea Description: A novel idea in wearable technology and security is the BioSecure.\nAbstract:']

--- LLM End ---
Response: [[ChatGeneration(text='**Abstract**\n\n**BioSecure: A Novel Wearable Technology for Enhanced Security**\n\n**Introduction:**\nWearable technology has emerged as a promising tool for various applications, including security. However, existing wearable security solutions often lack robustness and user convenience.\n\n**Methods:**\nThis paper presents BioSecure, a novel we


--- LLM End ---
Response: [[ChatGeneration(text='**Introduction**\n* Introduce the concept of wearable technology and its applications in security.\n* Highlight the limitations of existing wearable security solutions.\n* State the research gap and the significance of the proposed solution.\n\n**Related Work**\n* Review existing wearable security solutions, including their strengths and weaknesses.\n* Discuss biometric authentication techniques and their suitability for wearable devices.\n* Analyze encryption algorithms and their relevance to wearable security.\n\n**Problem Statement**\n* Clearly define the problem that BioSecure aims to address.\n* Explain the challenges and limitations of current approaches.\n* State the specific objectives of the proposed solution.\n\n**Proposed Methodology**\n* Describe the design and architecture of BioSecure.\n* Explain the process of capturing physiological signals and extracting biometric features.\n* Detail the machine learning algorithm used 